In [ ]:
pip install pillow

In [ ]:
from selenium import webdriver
from time import sleep
from bs4 import BeautifulSoup as bs
import requests as req
import os
import subprocess
import re
import json, pprint, random, webbrowser, pyautogui, pyperclip


In [ ]:
# 啟動瀏覽器
driver = webdriver.Chrome()
url = "https://www.gutenberg.org/browse/languages/zh"
driver.get(url)
driver.maximize_window()
res = req.get(url) 
soup = bs(res.text, "lxml") 

folderPath = 'novel'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)
    



In [ ]:
link_list = []
# 滾動頁面
def scroll():
    innerHeight = 0 #瀏覽器內部的高度
    offset = 0      #當前捲動的量(高度)
    count = 0       #累計無效滾動次數
    limit = 3       #最大無效滾動次數
    
    # 在捲動到沒有元素動態產生前，持續捲動
    while count <= limit:
        offset = driver.execute_script(
            'return document.documentElement.scrollHeight;'
        )
                
        # 捲軸往下滑動(第1個'{'是跳脫字元)
        driver.execute_script(f'''
            window.scrollTo({{
                top:{offset},
                behavior:"smooth" 
            }});
        ''') # behavior:"auto"是直接到底
        
        sleep(3)
        
        innerHeight = driver.execute_script(
            'return document.documentElement.scrollHeight;'
        )
        
        if offset == innerHeight:
            count += 1
        
        if offset >= 600:
            break

# 抓取title與連結            
def scrap_link():
    li_elements = soup.find_all("li", {"class": "pgdbetext"})[:300]
 
    
    
    for li_element in li_elements:
        if "(Chinese)" in li_element.text:
            a_element = li_element.find("a")
            href = a_element["href"]
            titleR = a_element.text
            title_list = re.findall(r'[\u4e00-\u9fff]+', titleR)
            
            if len(title_list) != 0: 
                title = ', '.join(title_list)+href[8:]
        
#         print(title)
    
                    
                link_list.append({
                    'title': title,
                    'link': href,
                    'content': ''    
                    })
            
    for link in link_list:
        link['link'] = 'https://www.gutenberg.org/files/' + link['link'][8:] + '/' + link['link'][8:] + '-0.txt'
    
#     pprint.pprint(link_list)
#     print(len(link_list))

# 走訪連結抓取內文產生檔案
def save_file():
    for index, link in enumerate(link_list[116:]):

        res_ = req.get(link_list[index+ 116]['link'])
        soup_ = bs(res_.content.decode('utf-8'), "lxml")

        
#         if index < 2:
#             print(soup_)
#             break
        if len(soup_) > 0:
            content_ = soup_.find('p').get_text()
            regexCh = r'[^\u4e00-\u9fff\u3002\uff1f\uff01\uff0c\u3001\uff1b\uff1a\u201c\u201d\u2018\u2019\uff08\uff09\u300a\u300b\u3008\u3009\u3010\u3011\u300e\u300f\u300c\u300d\ufe43\ufe44\u3014\u3015\u2026\u2014\uff5e\ufe4f\uffe5]+'
            contResult = re.sub(regexCh, '', content_)
            link_list[index+ 116]['content'] = contResult

        

        title = link_list[index+ 116]['title']
        with open(f"{folderPath}/{title}.txt", "w") as file:
            for link_list[index+ 116]['content'] in link_list:
                file.write(contResult)
        if (index + 116) % 10 == 0:
            sleep(10)
    

In [ ]:
if __name__ == "__main__":
#     scroll()
    scrap_link()
    save_file()

    
    

In [36]:

folder_path = r"G:\PYTHON\python_web_scraping\novel"
file_list = os.listdir(folder_path)
num_files = len(file_list)
print(f"資料夾裡總共有{num_files}個檔案")

#打開資料夾
os.startfile(folder_path)
webbrowser.open_new(folder_path)
sleep(2)
pyautogui.moveRel(-100, 0, 0.1) 
pyautogui.click()



file_names = os.listdir(folder_path)

for i in range(3):
    pyautogui.click()
    pyautogui.scroll(-500)
    selected_file = random.choice(file_names)
    file_path_ = os.path.join(folder_path, selected_file)
    
    webbrowser.open(file_path_)
    
    pyautogui.sleep(6)
    
    file_name = re.sub(r'\d+', '', os.path.splitext(selected_file)[0])
    
    pyperclip.copy(file_name)

    # 搜尋並反白內容
    pyautogui.hotkey('ctrl', 'f')
    pyautogui.hotkey('backspace')
    pyautogui.hotkey('ctrl', 'v')
    pyautogui.press('enter')
        
    sleep(2)


資料夾裡總共有244個檔案
